<a href="https://colab.research.google.com/github/vishakhasingh2003/5-ML_DS-FarmImpact/blob/main/RFR_Aggregate_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
file_id = '1KXMl4EVNofFnGsypt1UpZ4GJL6pGLJCe'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'maharashtra_aggregate_1966_2017.csv'
gdown.download(url, output, quiet=False)
import pandas as pd
df = pd.read_csv('maharashtra_aggregate_1966_2017.csv')

Downloading...
From: https://drive.google.com/uc?id=1KXMl4EVNofFnGsypt1UpZ4GJL6pGLJCe
To: /content/maharashtra_aggregate_1966_2017.csv
100%|██████████| 277k/277k [00:00<00:00, 14.2MB/s]


# Anu major 5

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r_squared = r2_score(y_true, y_pred)
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R^2': r_squared}

# Load your data
# Ensure your dataset has columns: 'Year', 'District', and other features
df = pd.read_csv('/content/Major_5_District.csv')

# Districts in Maharashtra
districts = ['Akola', 'Kolhapur', 'Ratnagiri', 'Pune', 'Wardha']

# Results dictionary to store evaluation metrics for each district
results = {}

# Loop through each district
for district in districts:
    print(f"Processing district: {district}")

    # Filter data for the district
    district_data = df[df['Dist Name'] == district].copy()

    # Drop non-predictive columns
    X = district_data.drop(columns=['Year', 'Dist Name', 'Production'])  # Replace 'Target_Variable' with your target column
    y = district_data['Production']

    # Normalize features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Train the Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predict on test data
    y_pred = rf.predict(X_test)

    # Calculate evaluation metrics
    metrics = calculate_metrics(y_test, y_pred)
    results[district] = metrics

    # Predict for the next 15 years
    future_years = np.arange(2018, 2018 + 15).reshape(-1, 1)
    future_features = np.tile(np.mean(X_scaled, axis=0), (15, 1))  # Example: Predict based on mean features
    future_predictions = rf.predict(future_features)

    print(f"Future Predictions for {district} (2018-2032):")
    print(future_predictions)

# Print evaluation metrics for all districts
print("\nEvaluation Metrics:")
for district, metrics in results.items():
    print(f"{district}: {metrics}")


Processing district: Akola
Future Predictions for Akola (2018-2032):
[1008.1551 1008.1551 1008.1551 1008.1551 1008.1551 1008.1551 1008.1551
 1008.1551 1008.1551 1008.1551 1008.1551 1008.1551 1008.1551 1008.1551
 1008.1551]
Processing district: Kolhapur
Future Predictions for Kolhapur (2018-2032):
[1362.6149 1362.6149 1362.6149 1362.6149 1362.6149 1362.6149 1362.6149
 1362.6149 1362.6149 1362.6149 1362.6149 1362.6149 1362.6149 1362.6149
 1362.6149]
Processing district: Ratnagiri
Future Predictions for Ratnagiri (2018-2032):
[392.2496 392.2496 392.2496 392.2496 392.2496 392.2496 392.2496 392.2496
 392.2496 392.2496 392.2496 392.2496 392.2496 392.2496 392.2496]
Processing district: Pune
Future Predictions for Pune (2018-2032):
[1038.8 1038.8 1038.8 1038.8 1038.8 1038.8 1038.8 1038.8 1038.8 1038.8
 1038.8 1038.8 1038.8 1038.8 1038.8]
Processing district: Wardha
Future Predictions for Wardha (2018-2032):
[411.5539 411.5539 411.5539 411.5539 411.5539 411.5539 411.5539 411.5539
 411.5539 411.

In [ ]:
# Updated Future Feature Projection
for district in districts:
    print(f"Processing district: {district}")

    # Filter data for the district
    district_data = df[df['Dist Name'] == district].copy()

    # Drop non-predictive columns and separate the target
    X = district_data.drop(columns=['Year', 'Dist Name', 'Production'])
    y = district_data['Production']

    # Add the Year column to features
    X['Year'] = district_data['Year']

    # Normalize features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Train the Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predict on test data
    y_pred = rf.predict(X_test)

    # Calculate evaluation metrics
    metrics = calculate_metrics(y_test, y_pred)
    results[district] = metrics

    # Predict for the next 15 years
    future_years = np.arange(2018, 2018 + 15)
    future_features = []

    # Calculate mean for numeric features only
    numeric_columns = X.select_dtypes(include=np.number).columns
    feature_means = X[numeric_columns].mean()

    for year in future_years:
        # Create future feature set
        future_row = feature_means.copy()
        future_row['Year'] = year  # Add future year
        future_features.append(future_row.values)

    future_features_scaled = scaler.transform(future_features)
    future_predictions = rf.predict(future_features_scaled)

    print(f"Future Predictions for {district} (2018-2032):")
    print(future_predictions)

# Print evaluation metrics for all districts
print("\nEvaluation Metrics:")
for district, metrics in results.items():
    print(f"{district}: {metrics}")


Processing district: Akola


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Future Predictions for Akola (2018-2032):
[1055.6823 1055.6823 1055.6823 1055.6823 1055.6823 1055.6823 1055.6823
 1055.6823 1055.6823 1055.6823 1055.6823 1055.6823 1055.6823 1055.6823
 1055.6823]
Processing district: Kolhapur


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Future Predictions for Kolhapur (2018-2032):
[1401.5725 1401.5725 1401.5725 1401.5725 1401.5725 1401.5725 1401.5725
 1401.5725 1401.5725 1401.5725 1401.5725 1401.5725 1401.5725 1401.5725
 1401.5725]
Processing district: Ratnagiri


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Future Predictions for Ratnagiri (2018-2032):
[390.4853 390.4853 390.4853 390.4853 390.4853 390.4853 390.4853 390.4853
 390.4853 390.4853 390.4853 390.4853 390.4853 390.4853 390.4853]
Processing district: Pune


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Future Predictions for Pune (2018-2032):
[1103.3607 1103.3607 1103.3607 1103.3607 1103.3607 1103.3607 1103.3607
 1103.3607 1103.3607 1103.3607 1103.3607 1103.3607 1103.3607 1103.3607
 1103.3607]
Processing district: Wardha
Future Predictions for Wardha (2018-2032):
[456.0208 456.0208 456.0208 456.0208 456.0208 456.0208 456.0208 456.0208
 456.0208 456.0208 456.0208 456.0208 456.0208 456.0208 456.0208]

Evaluation Metrics:
Akola: {'MSE': 83091.53122468188, 'RMSE': 288.2560168056894, 'MAE': 197.71070909090903, 'MAPE': 16.452965294504178, 'R^2': 0.7037176775570375}
Kolhapur: {'MSE': 26264.47984615634, 'RMSE': 162.06319707495697, 'MAE': 128.37399999999997, 'MAPE': 8.779063337105757, 'R^2': 0.9175010354287784}
Ratnagiri: {'MSE': 1088.6390978918168, 'RMSE': 32.99453133311363, 'MAE': 28.076209090909103, 'MAPE': 7.89394042603484, 'R^2': 0.914630336227309}
Pune: {'MSE': 56094.137137620004, 'RMSE': 236.8420088109793, 'MAE': 168.9374363636361, 'MAPE': 11.846702501844492, 'R^2': 0.8637989405928395}

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Function to calculate trends for numeric features
def calculate_feature_trends(data, numeric_columns):
    trends = {}
    for col in numeric_columns:
        # Perform linear regression to estimate trends
        years = data['Year'].values.reshape(-1, 1)
        values = data[col].values
        if len(np.unique(values)) > 1:  # Only calculate trends if there is variation
            coef = np.polyfit(years.flatten(), values, 1)  # Linear fit
            trends[col] = coef
        else:
            trends[col] = (0, values.mean())  # No trend, use mean
    return trends

# Generate future features based on trends
def generate_future_features(trends, future_years, numeric_columns):
    future_features = []
    for year in future_years:
        row = {}
        for col in numeric_columns:
            coef = trends[col]
            row[col] = coef[0] * year + coef[1]  # Linear projection: y = mx + c
        row['Year'] = year
        future_features.append(row)
    return pd.DataFrame(future_features)

# Updated Future Feature Projection
for district in districts:
    print(f"Processing district: {district}")

    # Filter data for the district
    district_data = df[df['Dist Name'] == district].copy()

    # Drop non-predictive columns and separate the target
    X = district_data.drop(columns=['Year', 'Dist Name', 'Production','Area'])
    y = district_data['Production']

    # Add the Year column to features
    X['Year'] = district_data['Year']

    # Normalize features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Train the Random Forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Predict on test data
    y_pred = rf.predict(X_test)

    # Calculate evaluation metrics
    metrics = calculate_metrics(y_test, y_pred)
    results[district] = metrics

    # Predict for the next 15 years
    future_years = np.arange(2018, 2018 + 15)
    numeric_columns = X.select_dtypes(include=np.number).columns
    trends = calculate_feature_trends(district_data, numeric_columns)
    future_features = generate_future_features(trends, future_years, numeric_columns)

    future_features_scaled = scaler.transform(future_features)
    future_predictions = rf.predict(future_features_scaled)

    print(f"Future Predictions for {district} (2018-2032):")
    print(future_predictions)

# Print evaluation metrics for all districts
print("\nEvaluation Metrics:")
for district, metrics in results.items():
    print(f"{district}: {metrics}")


Processing district: Akola
Future Predictions for Akola (2018-2032):
[1459.7053 1472.9156 1470.4449 1565.0148 1554.4451 1556.5132 1558.7853
 1546.703  1551.885  1540.7846 1537.3733 1532.8926 1560.9947 1551.6144
 1550.5015]
Processing district: Kolhapur
Future Predictions for Kolhapur (2018-2032):
[2239.4822 2241.6459 2241.6459 2241.699  2241.699  2241.699  2221.9634
 2221.9811 2223.3    2223.3    2223.3    2222.4729 2223.4514 2223.4514
 2223.4514]
Processing district: Ratnagiri
Future Predictions for Ratnagiri (2018-2032):
[465.5341 464.8023 465.2984 465.4673 465.4015 467.0825 467.6475 466.7925
 471.5715 471.5715 471.5038 471.5038 472.2618 472.2618 472.0132]
Processing district: Pune
Future Predictions for Pune (2018-2032):
[2050.5808 2058.3172 2076.7024 2068.3638 2076.7209 2081.2258 2079.8013
 2085.5867 2101.4142 2101.4142 2112.1108 2117.1648 2121.1405 2121.6058
 2121.6704]
Processing district: Wardha
Future Predictions for Wardha (2018-2032):
[540.4135 536.2156 532.6254 531.8762 529.

In [ ]:
The Gradient Boosting Regressor (GBR) algorithm is used here to predict agricultural attributes like production, yield, and climatic factors across different districts. GBR builds an ensemble of decision trees, where each tree is trained to correct the errors of its predecessor. This sequential approach makes it highly effective for complex, non-linear data patterns, which is typical in agricultural datasets that involve multiple features with varying relationships. GBR is particularly useful in this scenario because it can effectively handle small datasets and capture intricate patterns between variables such as temperature, precipitation, and fertilizer consumption. Its ability to model complex relationships allows for more accurate predictions in regions with varying agricultural conditions.

Fine-tuning for GBR is done using GridSearchCV, which tests multiple combinations of hyperparameters such as the number of estimators (trees), learning rate, and tree depth to find the optimal configuration for each district's data. This process improves model performance by reducing overfitting and enhancing prediction accuracy. After fine-tuning, the best model is used to generate predictions, which are then evaluated using metrics like R² and MSE to assess the model's accuracy. The fine-tuned GBR model is applied individually to each district, providing reliable predictions that account for regional variations in agricultural practices, thus assisting in long-term planning and decision-making for sustainable agricultural growth.

Processing district: Akola
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters for Akola: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None}
Future Predictions for Akola (2018-2032):
[1445.59942752 1468.82673202 1468.82673202 1476.97256535 1473.87752091
 1482.19542091 1487.48900395 1477.60001728 1477.60001728 1477.60001728
 1477.60001728 1467.49109061 1480.42632506 1480.42632506 1477.32810006]
Processing district: Kolhapur
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters for Kolhapur: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
Future Predictions for Kolhapur (2018-2032):
[2157.6431131 2157.6431131 2157.6431131 2157.6431131 2157.6431131
 2157.6431131 2143.0903931 2143.0903931 2143.0903931 2143.0903931
 2143.0903931 2143.0903931 2143.0903931 2143.0903931 2143.0903931]
Processing district: Ratnagiri
Fitting

# Vishakha 18 Jan

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# Load the dataset
file_path = '/content/maharashtra_aggregate_1966_2017.csv'  # Adjust path if needed
data = pd.read_csv(file_path)
data = data.drop(columns=['TOTAL AREA (1000 ha)', 'State Name'], errors='ignore')  # Adjust column names as necessary

# Define future years for forecasting
future_years = list(range(2018, 2026)) + [2030, 2035, 2040, 2045]

# Districts to process
districts = [
    "Ahmednagar", "Akola", "Amarawati", "Aurangabad", "Beed", "Bhandara",
    "Buldhana", "Chandrapur", "Dhule", "Jalgaon", "Kolhapur", "Nagpur",
    "Nanded", "Nasik", "Osmanabad", "Parbhani", "Pune", "Sangli",
    "Satara", "Solapur", "Yeotmal"
]

# Define target columns
target_columns = data.columns.drop(['Year', 'Dist Name'], errors='ignore')  # Exclude non-target columns


Define Hyperparameter Grid and Processing Function

In [ ]:
# Define hyperparameter tuning grid
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Function to process each district and target column
def process_district(district):
    results = {'fine_tuning': [], 'forecasting': []}
    district_data = data[data['Dist Name'] == district]

    for target_column in target_columns:
        # Prepare features (X) and target (y)
        X = district_data.drop(columns=['Year', 'Dist Name', target_column], errors='ignore')
        y = district_data[target_column]

        # Skip if insufficient data
        if len(y) < 5:
            continue

        # Train on historical data
        X_train = X
        y_train = y

        # Randomized search for hyperparameter tuning
        rf = RandomForestRegressor(random_state=42)
        random_search = RandomizedSearchCV(
            estimator=rf,
            param_distributions=param_distributions,
            n_iter=10,  # Reducing iterations for speed
            cv=2,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=0
        )
        random_search.fit(X_train, y_train)

        # Best parameters and model
        best_params = random_search.best_params_
        best_model = random_search.best_estimator_

        # Forecasting future years
        future_data = district_data.iloc[:len(future_years)].copy()
        future_data['Year'] = future_years
        future_X = future_data.drop(columns=['Year', 'Dist Name', target_column], errors='ignore')
        future_predictions = best_model.predict(future_X)

        # Store fine-tuning results
        mse = mean_squared_error(y_train, best_model.predict(X_train))
        r2 = r2_score(y_train, best_model.predict(X_train))
        results['fine_tuning'].append({
            'District': district,
            'Target Column': target_column,
            'Best Parameters': best_params,
            'MSE': mse,
            'R²': r2
        })

        # Store forecasted results
        for year, prediction in zip(future_years, future_predictions):
            results['forecasting'].append({
                'District': district,
                'Year': year,
                'Target Column': target_column,
                'Forecasted Value': prediction
            })

    return results


Run Parallel Processing for All Districts

In [ ]:
# Run processing in parallel
all_results = Parallel(n_jobs=-1)(delayed(process_district)(district) for district in districts)

# Combine results
fine_tuning_results = [item for result in all_results for item in result['fine_tuning']]
forecasted_results = [item for result in all_results for item in result['forecasting']]

# Save fine-tuning metrics to CSV
metrics_df = pd.DataFrame(fine_tuning_results)
metrics_csv_path = '/content/RF_fine_tuning_results.csv'
metrics_df.to_csv(metrics_csv_path, index=False)
print(f"Fine-tuning results saved to {metrics_csv_path}")

# Save forecasted results to CSV
forecasted_df = pd.DataFrame(forecasted_results)
forecasted_csv_path = '/content/RF_forecasted_future_values.csv'
forecasted_df.to_csv(forecasted_csv_path, index=False)
print(f"Forecasted values saved to {forecasted_csv_path}")


Fine-tuning results saved to /content/RF_fine_tuning_results.csv
Forecasted values saved to /content/RF_forecasted_future_values.csv


Aggregate Metrics and Save Summaries

In [ ]:
# Calculate aggregate fine-tuning metrics by district
aggregate_metrics = []
for district in metrics_df['District'].unique():
    district_metrics = metrics_df[metrics_df['District'] == district]

    avg_mse = district_metrics['MSE'].mean()
    avg_r2 = district_metrics['R²'].mean()

    aggregate_metrics.append({
        'District': district,
        'Avg MSE': avg_mse,
        'Avg R²': avg_r2
    })

# Save aggregated metrics to CSV
aggregate_metrics_df = pd.DataFrame(aggregate_metrics)
aggregate_metrics_csv_path = '/content/RF_aggregate_fine_tuning_metrics.csv'
aggregate_metrics_df.to_csv(aggregate_metrics_csv_path, index=False)
print(f"Aggregate fine-tuning metrics saved to {aggregate_metrics_csv_path}")

# Aggregate forecast summaries by district and year
forecast_summaries = []
for district in forecasted_df['District'].unique():
    district_forecasts = forecasted_df[forecasted_df['District'] == district]

    for year in district_forecasts['Year'].unique():
        year_forecasts = district_forecasts[district_forecasts['Year'] == year]

        avg_forecast = year_forecasts['Forecasted Value'].mean()

        forecast_summaries.append({
            'District': district,
            'Year': year,
            'Avg Forecasted Value': avg_forecast
        })

# Save forecast summaries to CSV
forecast_summary_df = pd.DataFrame(forecast_summaries)
forecast_summary_csv_path = '/content/RF_forecast_summary.csv'
forecast_summary_df.to_csv(forecast_summary_csv_path, index=False)
print(f"Forecast summaries saved to {forecast_summary_csv_path}")


Aggregate fine-tuning metrics saved to /content/RF_aggregate_fine_tuning_metrics.csv
Forecast summaries saved to /content/RF_forecast_summary.csv


Print Results

In [ ]:
# Print aggregate metrics
print("\nAggregate Fine-Tuning Metrics:")
print(aggregate_metrics_df)

# Print forecast summary
print("\nForecast Summary:")
print(forecast_summary_df)



Aggregate Fine-Tuning Metrics:
      District       Avg MSE    Avg R²
0   Ahmednagar  3.438887e+05  0.939631
1        Akola  1.946714e+05  0.943867
2    Amarawati  1.185667e+05  0.927274
3         Beed  1.497434e+05  0.909926
4     Bhandara  1.643366e+05  0.900408
5     Buldhana  1.511269e+05  0.943752
6   Chandrapur  8.567797e+04  0.934092
7        Dhule  8.495825e+04  0.931694
8      Jalgaon  4.536239e+04  0.935625
9     Kolhapur  1.066227e+05  0.926436
10      Nagpur  1.604328e+05  0.928035
11      Nanded  1.476730e+04  0.909378
12       Nasik  2.328904e+05  0.922410
13   Osmanabad  1.059048e+05  0.938908
14    Parbhani  2.344727e+04  0.934387
15        Pune  3.215764e+05  0.904351
16      Sangli  1.014062e+06  0.922289
17      Satara  7.896730e+04  0.932669
18     Solapur  1.175290e+05  0.919814
19     Yeotmal  1.282640e+05  0.922870

Forecast Summary:
       District  Year  Avg Forecasted Value
0    Ahmednagar  2018           1040.698694
1    Ahmednagar  2019           1247.20770